In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import *
from time import sleep
import prepare

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [2]:
# names = pd.read_csv('more_players.csv')

In [3]:
# names = names [['players']]

In [4]:
# names

In [5]:
# names[names.index==85]

In [6]:
# names

In [7]:
def get_puuid(name_list):
    puuid_list = []
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
    return puuid_list

In [8]:
# puuid_list = get_puuid(names.players)

In [9]:
# puuid_list

In [10]:
# puuid_df = pd.DataFrame(puuid_list)

In [11]:
# puuid_df.drop(columns = 'status', inplace=True)

In [12]:
# puuid_df.dropna(inplace=True)

In [13]:
# puuid_df.to_csv('puuid.csv',index=False)

In [14]:
# puuid = puuid_df[['puuid']]

In [3]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [16]:
#match_ids = pd.read_csv('match_ids.csv')

In [17]:
#match_ids = list(match_ids['0'])[1000:2000]

In [18]:
# df = get_match_info(match_ids, api_key, username, time=20)

In [19]:
#other_game_data = pd.read_json('other_game_data_bryant.json')

In [20]:
#timeline_data = pd.read_json('timeline_data_bryant.json')

In [21]:
#timeline_list = timeline_data.to_dict(orient = 'records')

In [22]:
#other_game_list = other_game_data.to_dict(orient = 'records')

In [23]:
# df = prepare(timeline_list, other_game_list, time=20)

In [24]:
# df.head()

In [25]:
# df.size

In [26]:
# df.shape

In [27]:
# df.columns

In [28]:
# df.to_csv('bryant_games.csv')

In [29]:
# df.winningTeam.value_counts()

In [4]:
#Load the match_ids.csv file
matches_df = pd.read_csv('match_ids.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
37279,NA1_4097058147
37280,NA1_4096721229
37281,NA1_4094049450
37282,NA1_4091691873


In [5]:
#Set up the match id list. You only need to uncomment the line following your name

match_ids = list(matches_df['0'])[10_657:17_314]

In [6]:
#Set up your username. You only need to uncomment the line following your name.
#Again, this username is only for naming the json files.

#Joshua Bryant
username = 'bryant'

In [7]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [8]:
#Before calling the get_match_info function, separate the match_ids list into smaller batches
match_batch_1 = match_ids[:1000]
match_batch_2 = match_ids[1000: 2000]
match_batch_3 = match_ids[2000: 3000]
match_batch_4 = match_ids[3000: 4000]
match_batch_5 = match_ids[4000: 5000]
match_batch_6 = match_ids[5000: 6000]
match_batch_7 = match_ids[6000: 6657]

In [35]:
#Since I will be running this function in smaller batches, I will have to update the username
#after each iteration. This will allow me to keep track of which json files contain what data

#For first iteration
username = 'start_10_657_end_11_657'

#Call the get_match_info function
df1 = get_match_info(match_batch_1, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

AttributeError: 'function' object has no attribute 'prepare'

In [37]:
jsonList = pd.read_json('timeline_data_start_10_657_end_11_657.json')
jsonList2 = pd.read_json('other_game_data_start_10_657_end_11_657.json')
t = 20

In [41]:
jsonList = jsonList.to_dict(orient='records')
jsonList2 = jsonList2.to_dict(orient='records')

In [42]:
df = prepare.prepare(jsonList, jsonList2, t)

Skipping: 0 due to <20 min or not classic
Finished with: 1 of 1000
Skipping: 2 due to <20 min or not classic
Finished with: 3 of 1000
Skipping: 4 due to <20 min or not classic
Skipping: 5 due to <20 min or not classic
Finished with: 6 of 1000
Finished with: 7 of 1000
Finished with: 8 of 1000
Finished with: 9 of 1000
Skipping: 10 due to <20 min or not classic
Skipping: 11 due to <20 min or not classic
Finished with: 12 of 1000
Finished with: 13 of 1000
Skipping: 14 due to <20 min or not classic
Finished with: 15 of 1000
Skipping: 16 due to <20 min or not classic
Finished with: 17 of 1000
Finished with: 18 of 1000
Finished with: 19 of 1000
Finished with: 20 of 1000
Finished with: 21 of 1000
Finished with: 22 of 1000
Finished with: 23 of 1000
Finished with: 24 of 1000
Finished with: 25 of 1000
Finished with: 26 of 1000
Finished with: 27 of 1000
Finished with: 28 of 1000
Finished with: 29 of 1000
Finished with: 30 of 1000
Finished with: 31 of 1000
Finished with: 32 of 1000
Finished with: 3

Finished with: 260 of 1000
Skipping: 261 due to <20 min or not classic
Finished with: 262 of 1000
Finished with: 263 of 1000
Finished with: 264 of 1000
Finished with: 265 of 1000
Finished with: 266 of 1000
Finished with: 267 of 1000
Skipping: 268 due to <20 min or not classic
Finished with: 269 of 1000
Finished with: 270 of 1000
Finished with: 271 of 1000
Skipping: 272 due to <20 min or not classic
Finished with: 273 of 1000
Finished with: 274 of 1000
Finished with: 275 of 1000
Finished with: 276 of 1000
Finished with: 277 of 1000
Skipping: 278 due to <20 min or not classic
Skipping: 279 due to <20 min or not classic
Finished with: 280 of 1000
Skipping: 281 due to <20 min or not classic
Skipping: 282 due to <20 min or not classic
Skipping: 283 due to <20 min or not classic
Finished with: 284 of 1000
Skipping: 285 due to <20 min or not classic
Finished with: 286 of 1000
Finished with: 287 of 1000
Skipping: 288 due to <20 min or not classic
Finished with: 289 of 1000
Skipping: 290 due to

Finished with: 517 of 1000
Finished with: 518 of 1000
Skipping: 519 due to <20 min or not classic
Finished with: 520 of 1000
Finished with: 521 of 1000
Skipping: 522 due to <20 min or not classic
Skipping: 523 due to <20 min or not classic
Finished with: 524 of 1000
Skipping: 525 due to <20 min or not classic
Finished with: 526 of 1000
Finished with: 527 of 1000
Finished with: 528 of 1000
Finished with: 529 of 1000
Finished with: 530 of 1000
Finished with: 531 of 1000
Finished with: 532 of 1000
Finished with: 533 of 1000
Finished with: 534 of 1000
Finished with: 535 of 1000
Finished with: 536 of 1000
Finished with: 537 of 1000
Finished with: 538 of 1000
Finished with: 539 of 1000
Finished with: 540 of 1000
Skipping: 541 due to <20 min or not classic
Finished with: 542 of 1000
Finished with: 543 of 1000
Skipping: 544 due to <20 min or not classic
Finished with: 545 of 1000
Finished with: 546 of 1000
Skipping: 547 due to <20 min or not classic
Skipping: 548 due to <20 min or not classic


Finished with: 774 of 1000
Finished with: 775 of 1000
Finished with: 776 of 1000
Finished with: 777 of 1000
Skipping: 778 due to <20 min or not classic
Skipping: 779 due to <20 min or not classic
Skipping: 780 due to <20 min or not classic
Finished with: 781 of 1000
Finished with: 782 of 1000
Finished with: 783 of 1000
Finished with: 784 of 1000
Finished with: 785 of 1000
Finished with: 786 of 1000
Finished with: 787 of 1000
Skipping: 788 due to <20 min or not classic
Finished with: 789 of 1000
Skipping: 790 due to <20 min or not classic
Skipping: 791 due to <20 min or not classic
Finished with: 792 of 1000
Finished with: 793 of 1000
Skipping: 794 due to <20 min or not classic
Finished with: 795 of 1000
Finished with: 796 of 1000
Finished with: 797 of 1000
Finished with: 798 of 1000
Finished with: 799 of 1000
Skipping: 800 due to <20 min or not classic
Finished with: 801 of 1000
Finished with: 802 of 1000
Finished with: 803 of 1000
Finished with: 804 of 1000
Finished with: 805 of 1000


In [44]:
df.to_csv('bryant_start_10_657_end_11_657.csv')

In [45]:
#For second iteration
username = 'start_11_657_end_12_657'

#Call the get_match_info function
df2 = get_match_info(match_batch_2, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

Finished with: 213 of 1000
Skipping: 214 due to <20 min or not classic
Finished with: 215 of 1000
Skipping: 216 due to <20 min or not classic
Finished with: 217 of 1000
Skipping: 218 due to <20 min or not classic
Skipping: 219 due to <20 min or not classic
Finished with: 220 of 1000
Finished with: 221 of 1000
Finished with: 222 of 1000
Finished with: 223 of 1000
Finished with: 224 of 1000
Finished with: 225 of 1000
Finished with: 226 of 1000
Finished with: 227 of 1000
Finished with: 228 of 1000
Finished with: 229 of 1000
Finished with: 230 of 1000
Finished with: 231 of 1000
Finished with: 232 of 1000
Finished with: 233 of 1000
Finished with: 234 of 1000
Finished with: 235 of 1000
Finished with: 236 of 1000
Finished with: 237 of 1000
Finished with: 238 of 1000
Finished with: 239 of 1000
Finished with: 240 of 1000
Finished with: 241 of 1000
Finished with: 242 of 1000
Skipping: 243 due to <20 min or not classic
Finished with: 244 of 1000
Finished with: 245 of 1000
Finished with: 246 of 10

Finished with: 475 of 1000
Skipping: 476 due to <20 min or not classic
Finished with: 477 of 1000
Skipping: 478 due to <20 min or not classic
Skipping: 479 due to <20 min or not classic
Finished with: 480 of 1000
Finished with: 481 of 1000
Skipping: 482 due to <20 min or not classic
Skipping: 483 due to <20 min or not classic
Finished with: 484 of 1000
Finished with: 485 of 1000
Finished with: 486 of 1000
Finished with: 487 of 1000
Finished with: 488 of 1000
Finished with: 489 of 1000
Skipping: 490 due to <20 min or not classic
Finished with: 491 of 1000
Finished with: 492 of 1000
Finished with: 493 of 1000
Finished with: 494 of 1000
Finished with: 495 of 1000
Finished with: 496 of 1000
Finished with: 497 of 1000
Finished with: 498 of 1000
Finished with: 499 of 1000
Finished with: 500 of 1000
Finished with: 501 of 1000
Finished with: 502 of 1000
Finished with: 503 of 1000
Finished with: 504 of 1000
Skipping: 505 due to <20 min or not classic
Finished with: 506 of 1000
Finished with: 50

Finished with: 734 of 1000
Finished with: 735 of 1000
Skipping: 736 due to <20 min or not classic
Finished with: 737 of 1000
Finished with: 738 of 1000
Finished with: 739 of 1000
Finished with: 740 of 1000
Finished with: 741 of 1000
Skipping: 742 due to <20 min or not classic
Skipping: 743 due to <20 min or not classic
Skipping: 744 due to <20 min or not classic
Skipping: 745 due to <20 min or not classic
Skipping: 746 due to <20 min or not classic
Finished with: 747 of 1000
Finished with: 748 of 1000
Skipping: 749 due to <20 min or not classic
Finished with: 750 of 1000
Finished with: 751 of 1000
Skipping: 752 due to <20 min or not classic
Finished with: 753 of 1000
Finished with: 754 of 1000
Finished with: 755 of 1000
Finished with: 756 of 1000
Skipping: 757 due to <20 min or not classic
Finished with: 758 of 1000
Finished with: 759 of 1000
Skipping: 760 due to <20 min or not classic
Skipping: 761 due to <20 min or not classic
Finished with: 762 of 1000
Skipping: 763 due to <20 min o

Finished with: 989 of 1000
Skipping: 990 due to <20 min or not classic
Finished with: 991 of 1000
Finished with: 992 of 1000
Skipping: 993 due to <20 min or not classic
Finished with: 994 of 1000
Skipping: 995 due to <20 min or not classic
Finished with: 996 of 1000
Skipping: 997 due to <20 min or not classic
Finished with: 998 of 1000
Finished with: 999 of 1000


In [47]:
df2.to_csv('bryant_start_11_657_end_12_657')

In [9]:
#For third iteration
username = 'start_12_657_end_13_657'

#Call the get_match_info function
df3 = get_match_info(match_batch_3, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

Finished with: 208 of 1000
Finished with: 209 of 1000
Finished with: 210 of 1000
Skipping: 211 due to <20 min or not classic
Finished with: 212 of 1000
Skipping: 213 due to <20 min or not classic
Finished with: 214 of 1000
Skipping: 215 due to <20 min or not classic
Finished with: 216 of 1000
Finished with: 217 of 1000
Skipping: 218 due to <20 min or not classic
Finished with: 219 of 1000
Finished with: 220 of 1000
Finished with: 221 of 1000
Finished with: 222 of 1000
Finished with: 223 of 1000
Skipping: 224 due to <20 min or not classic
Skipping: 225 due to <20 min or not classic
Finished with: 226 of 1000
Finished with: 227 of 1000
Finished with: 228 of 1000
Finished with: 229 of 1000
Finished with: 230 of 1000
Finished with: 231 of 1000
Finished with: 232 of 1000
Finished with: 233 of 1000
Finished with: 234 of 1000
Finished with: 235 of 1000
Finished with: 236 of 1000
Finished with: 237 of 1000
Skipping: 238 due to <20 min or not classic
Finished with: 239 of 1000
Finished with: 24

Finished with: 470 of 1000
Skipping: 471 due to <20 min or not classic
Finished with: 472 of 1000
Skipping: 473 due to <20 min or not classic
Skipping: 474 due to <20 min or not classic
Skipping: 475 due to <20 min or not classic
Skipping: 476 due to <20 min or not classic
Finished with: 477 of 1000
Skipping: 478 due to <20 min or not classic
Finished with: 479 of 1000
Finished with: 480 of 1000
Skipping: 481 due to <20 min or not classic
Skipping: 482 due to <20 min or not classic
Finished with: 483 of 1000
Finished with: 484 of 1000
Skipping: 485 due to <20 min or not classic
Finished with: 486 of 1000
Skipping: 487 due to <20 min or not classic
Skipping: 488 due to <20 min or not classic
Skipping: 489 due to <20 min or not classic
Finished with: 490 of 1000
Finished with: 491 of 1000
Skipping: 492 due to <20 min or not classic
Finished with: 493 of 1000
Finished with: 494 of 1000
Skipping: 495 due to <20 min or not classic
Skipping: 496 due to <20 min or not classic
Skipping: 497 du

Finished with: 727 of 1000
Finished with: 728 of 1000
Finished with: 729 of 1000
Finished with: 730 of 1000
Finished with: 731 of 1000
Skipping: 732 due to <20 min or not classic
Skipping: 733 due to <20 min or not classic
Finished with: 734 of 1000
Finished with: 735 of 1000
Finished with: 736 of 1000
Finished with: 737 of 1000
Finished with: 738 of 1000
Finished with: 739 of 1000
Finished with: 740 of 1000
Finished with: 741 of 1000
Finished with: 742 of 1000
Finished with: 743 of 1000
Finished with: 744 of 1000
Finished with: 745 of 1000
Finished with: 746 of 1000
Skipping: 747 due to <20 min or not classic
Finished with: 748 of 1000
Finished with: 749 of 1000
Finished with: 750 of 1000
Finished with: 751 of 1000
Skipping: 752 due to <20 min or not classic
Finished with: 753 of 1000
Skipping: 754 due to <20 min or not classic
Finished with: 755 of 1000
Skipping: 756 due to <20 min or not classic
Finished with: 757 of 1000
Finished with: 758 of 1000
Finished with: 759 of 1000
Finishe

Finished with: 990 of 1000
Finished with: 991 of 1000
Finished with: 992 of 1000
Finished with: 993 of 1000
Skipping: 994 due to <20 min or not classic
Skipping: 995 due to <20 min or not classic
Finished with: 996 of 1000
Skipping: 997 due to <20 min or not classic
Finished with: 998 of 1000
Finished with: 999 of 1000


In [10]:
df3.to_csv('bryant_start_12_657_end_13_657')

In [11]:
#For the fourth iteration
username = 'start_13_657_end_14_657'

#Call the get_match_info function
df4 = get_match_info(match_batch_4, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

Finished with: 212 of 1000
Skipping: 213 due to <20 min or not classic
Finished with: 214 of 1000
Skipping: 215 due to <20 min or not classic
Skipping: 216 due to <20 min or not classic
Finished with: 217 of 1000
Skipping: 218 due to <20 min or not classic
Skipping: 219 due to <20 min or not classic
Skipping: 220 due to <20 min or not classic
Finished with: 221 of 1000
Finished with: 222 of 1000
Finished with: 223 of 1000
Skipping: 224 due to <20 min or not classic
Finished with: 225 of 1000
Finished with: 226 of 1000
Finished with: 227 of 1000
Finished with: 228 of 1000
Finished with: 229 of 1000
Finished with: 230 of 1000
Finished with: 231 of 1000
Skipping: 232 due to <20 min or not classic
Finished with: 233 of 1000
Finished with: 234 of 1000
Finished with: 235 of 1000
Finished with: 236 of 1000
Finished with: 237 of 1000
Finished with: 238 of 1000
Skipping: 239 due to <20 min or not classic
Finished with: 240 of 1000
Finished with: 241 of 1000
Skipping: 242 due to <20 min or not c

Finished with: 470 of 1000
Finished with: 471 of 1000
Skipping: 472 due to <20 min or not classic
Skipping: 473 due to <20 min or not classic
Finished with: 474 of 1000
Finished with: 475 of 1000
Finished with: 476 of 1000
Finished with: 477 of 1000
Finished with: 478 of 1000
Finished with: 479 of 1000
Skipping: 480 due to <20 min or not classic
Finished with: 481 of 1000
Finished with: 482 of 1000
Skipping: 483 due to <20 min or not classic
Skipping: 484 due to <20 min or not classic
Skipping: 485 due to <20 min or not classic
Finished with: 486 of 1000
Finished with: 487 of 1000
Finished with: 488 of 1000
Skipping: 489 due to <20 min or not classic
Finished with: 490 of 1000
Finished with: 491 of 1000
Skipping: 492 due to <20 min or not classic
Finished with: 493 of 1000
Finished with: 494 of 1000
Finished with: 495 of 1000
Finished with: 496 of 1000
Skipping: 497 due to <20 min or not classic
Skipping: 498 due to <20 min or not classic
Finished with: 499 of 1000
Finished with: 500 o

Finished with: 729 of 1000
Skipping: 730 due to <20 min or not classic
Finished with: 731 of 1000
Finished with: 732 of 1000
Skipping: 733 due to <20 min or not classic
Finished with: 734 of 1000
Finished with: 735 of 1000
Finished with: 736 of 1000
Skipping: 737 due to <20 min or not classic
Finished with: 738 of 1000
Finished with: 739 of 1000
Finished with: 740 of 1000
Finished with: 741 of 1000
Finished with: 742 of 1000
Skipping: 743 due to <20 min or not classic
Finished with: 744 of 1000
Finished with: 745 of 1000
Skipping: 746 due to <20 min or not classic
Skipping: 747 due to <20 min or not classic
Finished with: 748 of 1000
Finished with: 749 of 1000
Finished with: 750 of 1000
Finished with: 751 of 1000
Finished with: 752 of 1000
Finished with: 753 of 1000
Finished with: 754 of 1000
Finished with: 755 of 1000
Skipping: 756 due to <20 min or not classic
Finished with: 757 of 1000
Skipping: 758 due to <20 min or not classic
Finished with: 759 of 1000
Skipping: 760 due to <20 mi

Finished with: 992 of 1000
Finished with: 993 of 1000
Skipping: 994 due to <20 min or not classic
Finished with: 995 of 1000
Finished with: 996 of 1000
Skipping: 997 due to <20 min or not classic
Finished with: 998 of 1000
Skipping: 999 due to <20 min or not classic


In [12]:
df4.to_csv('bryant_start_13_657_end_14_657')

In [13]:
#For the fifth iteration
username = 'start_14_657_end_15_657'

#Call the get_match_info function
df5 = get_match_info(match_batch_5, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Something went wrong getting OTHER GAME DATA! Status Code 404 for match ID: NA1_4068659096.
Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Complet

Finished with: 216 of 999
Finished with: 217 of 999
Finished with: 218 of 999
Skipping: 219 due to <20 min or not classic
Finished with: 220 of 999
Finished with: 221 of 999
Finished with: 222 of 999
Finished with: 223 of 999
Finished with: 224 of 999
Finished with: 225 of 999
Skipping: 226 due to <20 min or not classic
Finished with: 227 of 999
Finished with: 228 of 999
Finished with: 229 of 999
Finished with: 230 of 999
Finished with: 231 of 999
Finished with: 232 of 999
Finished with: 233 of 999
Finished with: 234 of 999
Finished with: 235 of 999
Skipping: 236 due to <20 min or not classic
Finished with: 237 of 999
Finished with: 238 of 999
Finished with: 239 of 999
Finished with: 240 of 999
Finished with: 241 of 999
Finished with: 242 of 999
Finished with: 243 of 999
Finished with: 244 of 999
Finished with: 245 of 999
Finished with: 246 of 999
Skipping: 247 due to <20 min or not classic
Finished with: 248 of 999
Finished with: 249 of 999
Finished with: 250 of 999
Skipping: 251 due 

Finished with: 481 of 999
Finished with: 482 of 999
Finished with: 483 of 999
Finished with: 484 of 999
Skipping: 485 due to <20 min or not classic
Finished with: 486 of 999
Finished with: 487 of 999
Finished with: 488 of 999
Finished with: 489 of 999
Skipping: 490 due to <20 min or not classic
Finished with: 491 of 999
Skipping: 492 due to <20 min or not classic
Skipping: 493 due to <20 min or not classic
Finished with: 494 of 999
Finished with: 495 of 999
Finished with: 496 of 999
Finished with: 497 of 999
Finished with: 498 of 999
Finished with: 499 of 999
Finished with: 500 of 999
Finished with: 501 of 999
Finished with: 502 of 999
Skipping: 503 due to <20 min or not classic
Finished with: 504 of 999
Skipping: 505 due to <20 min or not classic
Finished with: 506 of 999
Finished with: 507 of 999
Finished with: 508 of 999
Finished with: 509 of 999
Finished with: 510 of 999
Finished with: 511 of 999
Finished with: 512 of 999
Finished with: 513 of 999
Finished with: 514 of 999
Finished

Finished with: 757 of 999
Skipping: 758 due to <20 min or not classic
Finished with: 759 of 999
Finished with: 760 of 999
Finished with: 761 of 999
Finished with: 762 of 999
Finished with: 763 of 999
Skipping: 764 due to <20 min or not classic
Skipping: 765 due to <20 min or not classic
Skipping: 766 due to <20 min or not classic
Finished with: 767 of 999
Skipping: 768 due to <20 min or not classic
Finished with: 769 of 999
Finished with: 770 of 999
Finished with: 771 of 999
Finished with: 772 of 999
Finished with: 773 of 999
Finished with: 774 of 999
Finished with: 775 of 999
Finished with: 776 of 999
Finished with: 777 of 999
Skipping: 778 due to <20 min or not classic
Finished with: 779 of 999
Finished with: 780 of 999
Finished with: 781 of 999
Finished with: 782 of 999
Finished with: 783 of 999
Skipping: 784 due to <20 min or not classic
Finished with: 785 of 999
Finished with: 786 of 999
Skipping: 787 due to <20 min or not classic
Finished with: 788 of 999
Finished with: 789 of 99

In [14]:
df5.to_csv('bryant_start_14_657_end_15_657')